# Zhang's Algorithm For Camera Calibration

### Import Statements

In [1]:
import os
from camera_callibration_helper import *
import cv2
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from copy import deepcopy

### Load the Images
* raw_img_list (list): list of 40 BGR input images
* grey_img_list (list): list of 40 grey scale input images

In [2]:
given_data_path = "/home/jo_wang/Desktop/ECE661/HW08/Dataset1"
raw_img_list, grey_img_list = loadImages(given_data_path)
assert(len(grey_img_list) == 40)
assert(len(raw_img_list) == 40)

### Apply Canny Edge Detector On Grey Scale Images
* edge_img_list (list): list of edge maps from Canny

In [3]:
edge_img_list = performCanny(grey_img_list)
assert(len(edge_img_list) == 40)

In [4]:
hough_lines_list = performHoughTransform(edge_img_list)
assert(len(hough_lines_list) == len(edge_img_list))

In [17]:
h_lines, v_lines = get_Horizontal_Vert_Lines(hough_lines_list[33])

v_lines = np.array(v_lines).reshape(-1,2)
h_lines = np.array(h_lines).reshape(-1,2)

x_intercept = list()
for i in range(v_lines.shape[0]):
    rho, theta = v_lines[i]
    x_intercept.append(np.divide(rho, np.cos(theta)))

y_intercept = list()
for i in range(h_lines.shape[0]):
    rho, theta = h_lines[i]
    y_intercept.append(np.divide(rho, np.sin(theta)))


assert(len(x_intercept) == len(v_lines))
assert(len(y_intercept) == len(h_lines))

kmeans_v_lines = KMeans(n_clusters=8, random_state=0).fit(np.array(x_intercept).reshape(-1,1))
kmeans_h_lines = KMeans(n_clusters=10, random_state=0).fit(np.array(y_intercept).reshape(-1,1))

v_clustered_lines = list()
h_clustered_lines = list()

for i in range(8):
    v_clustered_lines.append(list(np.mean(v_lines[kmeans_v_lines.labels_ == i], axis=0)))

for i in range(10):
    h_clustered_lines.append(list(np.mean(h_lines[kmeans_h_lines.labels_ == i], axis=0)))

v_lines_sorted = sorted(v_clustered_lines, key=lambda x: np.abs(x[0]/x[1]))
h_lines_sorted = sorted(h_clustered_lines, key=lambda x: np.abs(x[0]/x[1]))


corner_points = list()
final_v_lines = list()
final_h_lines = list()
for v_line in v_lines_sorted:
    v_rho, v_theta = v_line
    v_HC = np.array([np.cos(v_theta), np.sin(v_theta), -v_rho])
    final_h_lines.append(v_HC)
    for h_line in h_lines_sorted:
        h_rho, h_theta = h_line
        h_HC = np.array([np.cos(h_theta), np.sin(h_theta), -h_rho])
        if i == 1:
            final_h_lines.append(h_HC)
        point = np.cross(h_HC, v_HC)
        point = point / point[-1]
        corner_points.append(tuple(point[:2].astype('int')))

img = deepcopy(raw_img_list[33])
for i, point in enumerate(corner_points):
    img = cv2.circle(img, point, 6, (0, 0, 255), -1)
    cv2.putText(img, str(i), (point[0]-5, point[1]+5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (246,255,12), 1)

cv2.imwrite("points.jpg", img)






(17, 2)
[0 4 7 6 1 3 2 5 5 2 1 3 0 6 4 2 7]
[5 8 4 7 3 0 2 6 1 9 4 2 0 5 6]


True

In [18]:
world_points = list()
for i in range(0, 200, 20):
    for j in range(0, 160, 20):
        world_points.append([i,j])
assert(len(world_points) == 80)